# Setup

In [2]:
from lets_plot import *
import pandas as pd
import sqlite3 as sql

In [3]:
db = sql.connect('lahmansbaseballdb.sqlite')

In [4]:
q0 = '''
select *
from allstarfull
limit 5
'''

In [5]:
pd.read_sql_query(q0,db)

,ID,playerID,yearID,gameNum,gameID,teamID,team_ID,lgID,GP,startingPos
0,1,gomezle01,1933,0,ALS193307060,NYA,921,AL,1,1
1,2,ferreri01,1933,0,ALS193307060,BOS,912,AL,1,2
2,3,gehrilo01,1933,0,ALS193307060,NYA,921,AL,1,3
3,4,gehrich01,1933,0,ALS193307060,DET,919,AL,1,4
4,5,dykesji01,1933,0,ALS193307060,CHA,915,AL,1,5


# Q1

In [7]:
q1 = '''
select distinct cp.playerID, cp.schoolID, sal.salary, sal.yearID, sal.teamID
from collegeplaying cp
join schools sch on cp.schoolID = sch.schoolID
join salaries sal on cp.playerID = sal.playerID
where sch.schoolID = 'idbyuid'
order by sal.salary desc;
'''

In [8]:
pd.read_sql_query(q1, db)

,playerID,schoolID,salary,yearID,teamID
0,lindsma01,idbyuid,4000000.0,2014,CHA
1,lindsma01,idbyuid,3600000.0,2012,BAL
2,lindsma01,idbyuid,2800000.0,2011,COL
3,lindsma01,idbyuid,2300000.0,2013,CHA
4,lindsma01,idbyuid,1625000.0,2010,HOU
5,stephga01,idbyuid,1025000.0,2001,SLN
6,stephga01,idbyuid,900000.0,2002,SLN
7,stephga01,idbyuid,800000.0,2003,SLN
8,stephga01,idbyuid,550000.0,2000,SLN
9,lindsma01,idbyuid,410000.0,2009,FLO


# Q2

In [4]:
q2_1 = '''
select playerID, yearID, round(cast(H as float) / AB, 3) as batting_average
from batting
where AB > 0
order by batting_average desc, playerID asc
limit 5;
'''

In [28]:
# Just checking that the first few people do actually have a 100% batting average: 1/1=1, so yes.
# pd.read_sql_query("select playerID, yearID, AB, H from batting where AB > 0 and playerID in ('aberal01', 'abernte02', 'abramge01')", db)

In [31]:
pd.read_sql_query(q2_1, db)

,playerID,yearID,batting_average
0,aberal01,1957,1.0
1,abernte02,1960,1.0
2,abramge01,1923,1.0
3,acklefr01,1964,1.0
4,alanirj01,2019,1.0


In [ ]:
q2_2 = q2_1.replace('AB > 0', 'AB > 9')

In [33]:
pd.read_sql_query(q2_2, db)

,playerID,yearID,batting_average
0,nymanny01,1974,0.643
1,carsoma01,2013,0.636
2,silvech01,1948,0.571
3,puccige01,1930,0.563
4,applepe01,1927,0.545


In [45]:
q2_3 =  q2_1.replace('yearID, ', '')\
            .replace('H ', 'sum(H) ')\
            .replace('AB,', 'sum(AB),')\
            .replace('batting\n','batting\ngroup by playerID\n')\
            .replace('where AB > 0', 'having sum(AB) > 100')

In [46]:
print(q2_3)


select playerID, round(cast(sum(H) as float) / sum(AB), 3) as batting_average
from batting
group by playerID
having sum(AB) > 100
order by batting_average desc, playerID asc
limit 5;



In [47]:
pd.read_sql_query(q2_3, db)

,playerID,batting_average
0,cobbty01,0.366
1,barnero01,0.360
2,hornsro01,0.358
3,jacksjo01,0.356
4,meyerle01,0.356


# Q3

In [48]:
q3 = '''
select 
    yearID,
    sum(case when teamID = 'NYA' then HR else 0 end) as yankees_hr,
    sum(case when teamID = 'BOS' then HR else 0 end) as redsox_hr
from teams
where teamID in ('NYA', 'BOS')
group by yearID
having sum(case when teamID = 'NYA' then HR else 0 end) > 0
   OR sum(case when teamID = 'BOS' then HR else 0 end) > 0
order by yearID;
'''

In [67]:
df = pd.read_sql_query(q3, db)
df

,yearID,yankees_hr,redsox_hr
0,1901,0,37
1,1902,0,42
2,1903,18,48
3,1904,27,26
4,1905,23,29
...,...,...,...
114,2015,212,161
115,2016,183,208
116,2017,241,168
117,2018,267,208


In [77]:
LetsPlot.setup_html()

df['hr_diff'] = df['yankees_hr'] - df['redsox_hr']

# Get min/max for full rectangle coverage
x_min = df['yearID'].min() - 5
x_max = df['yearID'].max() + 5
y_min = df['hr_diff'].min() - 10
y_max = df['hr_diff'].max() + 10

ggplot(df, aes(x='yearID', y='hr_diff')) + \
    geom_rect(xmin=x_min, xmax=x_max, ymin=y_min, ymax=0, fill='rgba(26,43,78,.8)', inherit_aes=False) + \
    geom_rect(xmin=x_min, xmax=x_max, ymin=0, ymax=y_max, fill='rgba(221,51,61,.8)', inherit_aes=False) + \
    geom_line(color='white') + \
    geom_hline(yintercept=0, linetype='dashed', color='white') + \
    ggtitle('Yankees vs Red Sox: Yearly Home Run Difference') + \
    xlab('Year') + \
    ylab('HR Difference\nYankees - Red Sox') + \
    scale_x_continuous(format='d')

# SQ1

In [78]:
sq1 = '''
with primary_positions as (
    select playerID, yearID, POS,
           row_number() over (
               partition by playerID, yearID
               order by sum(G) desc
           ) as pos_rank
    from fielding
    group by playerID, yearID, POS
),
main_position as (
    select playerID, yearID, POS
    from primary_positions
    where pos_rank = 1
),
position_salaries as (
    select 
        mp.POS as position,
        s.salary,
        s.playerID
    from main_position mp
    join salaries s 
        ON mp.playerID = s.playerID AND mp.yearID = s.yearID
)
select 
    position,
    round(avg(salary), 2) as average_salary,
    count(distinct playerID) as total_players,
    max(salary) as highest_salary,
    case 
        when avg(salary) > 3000000 then 'High Salary'
        when avg(salary) between 2000000 AND 3000000 then 'Medium Salary'
        else 'Low Salary'
    end as salary_category
from position_salaries
group by position
order by average_salary desc;
'''

In [79]:
pd.read_sql_query(sq1, db)

,position,average_salary,total_players,highest_salary,salary_category
0,1B,3380780.30,454,28000000.0,High Salary
1,OF,2396919.04,1128,27328046.0,Medium Salary
2,3B,2323326.28,490,33000000.0,Medium Salary
3,SS,1973603.73,377,22600000.0,Low Salary
4,P,1938130.80,2556,33000000.0,Low Salary
5,2B,1821073.90,478,24000000.0,Low Salary
6,C,1430337.51,403,23000000.0,Low Salary


# SQ2

In [ ]:
sq2 = '''
with games_per_player_year as (
    select playerID, yearID, sum(G_all) as total_games
    from appearances
    group by playerID, yearID
),
qualified_players as (
    select playerID
    from games_per_player_year
    group by playerID
    having sum(total_games) >= 10
),
career_span as (
    select 
        qp.playerID,
        min(gpy.yearID) as first_year,
        max(gpy.yearID) as last_year,
        max(gpy.yearID) - min(gpy.yearID) as career_length
    from qualified_players qp
    join games_per_player_year gpy on qp.playerID = gpy.playerID
    group by qp.playerID
),
top_10_longest_careers as (
    select *
    from career_span
    order by career_length desc
    limit 10
)
select 
    c.playerID,
    p.nameFirst as first_name,
    p.nameLast as last_name,
    c.career_length
from top_10_longest_careers c
join people p on c.playerID = p.playerID
order by c.career_length desc;
'''

In [85]:
pd.read_sql_query(sq2, db)

,playerID,first_name,last_name,career_length
0,altroni01,Nick,Altrock,35
1,orourji01,Jim,O'Rourke,32
2,minosmi01,Minnie,Minoso,31
3,olearch01,Charley,O'Leary,30
4,lathaar01,Arlie,Latham,29
5,mcguide01,Deacon,McGuire,28
6,eversjo01,Johnny,Evers,27
7,jennihu01,Hughie,Jennings,27
8,ryanno01,Nolan,Ryan,27
9,streega01,Gabby,Street,27


# Quiz Questions

In [5]:
qq1 = q2_1.replace('AB > 0', 'playerID = \'addybo01\'')

In [6]:
pd.read_sql_query(qq1, db)

,playerID,yearID,batting_average
0,addybo01,1873,0.355
1,addybo01,1873,0.314
2,addybo01,1876,0.282
3,addybo01,1877,0.278
4,addybo01,1871,0.271
